In [ ]:
import requests
# from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
import re
import time as t
import urllib
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry


In [ ]:
import requests
from bs4 import BeautifulSoup

urls = []
month = []
day = []
year = []
dj = []

headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for page in range(1,35):
   
    url = 'https://spinitron.com/KDHX/show/126906/Sound-Salvation?page={}'.format(page)
    reqs = requests.get(url, headers=headers)
    
    
    rock_soup = BeautifulSoup(reqs.text, 'html.parser')
    rock_soup.prettify()
    
    for link in rock_soup.find_all('a'):
        urls.append(link.get('href'))
        


In [ ]:
# urls = [pl['href'] for pl in urls if "/pl/" in pl['href']]
urls = [url for url in urls if "/pl/" in url]

In [ ]:

print(len(urls))


In [ ]:
#drop duplicates
sorted_urls = list(set(urls))

In [ ]:
print(len(sorted_urls))

In [ ]:
# sorted_urls = sorted(sorted_urls, key=lambda x: x.split('/')[-1])
# sorted_urls

In [ ]:
len(sorted_urls) == len(set(sorted_urls))

In [ ]:
# # grab the month, day, year, and dj for each show
# url = 'https://spinitron.com/' + sorted_urls[0]
# # reqs = requests.get(url, headers=headers)
# # rock_soup = BeautifulSoup(reqs.text, 'html.parser')
# # rock_soup.prettify()
# # month.append(rock_soup.find('span', {'class': 'month'}).text)
# # day.append(rock_soup.find('span', {'class': 'day'}).text)
# # year.append(rock_soup.find('span', {'class': 'year'}).text)
# # dj.append(rock_soup.find('span', {'class': 'dj'}).text)
# # t.sleep(1)

# month.append(rock_soup.find_all("span", "month"))
# day.append(rock_soup.find_all("span", "day"))
# year.append(rock_soup.find_all("span", "year"))
# # dj.append(rock_soup.find_all("div", "dj"))

# dictionary = {
    
# 'urls': url, 
# 'month': month, 
# 'day': day, 
# 'year': year, 
# # 'dj': dj}
# }

# date_and_dj = pd.DataFrame(dictionary)

# date_and_dj

In [ ]:
#make a function that takes in a list of urls and returns a dataframe of all the playlists
def sound_salvation_spider(u):
    """Perform web scraping for any sound salvation playlist given the available link"""

    #get the page
    r = requests.get(u)
    rock_soup = BeautifulSoup(r.text, 'html')

    #get the data
    artist = rock_soup.find_all("span", "artist")
    song = rock_soup.find_all("span", "song")
    album = rock_soup.find_all("span", "release")
    album_label = rock_soup.find_all("span", "label")
    album_year = rock_soup.find_all("span", "released")
    time = rock_soup.find_all("td", "spin-time")
    note = rock_soup.find_all('div', attrs={'class':'note'})
    url = u
    

    # append the data to the empty lists
    artists = ([a.string for a in artist])
    songs = ([s.string for s in song])
    albums = ([a.string for a in album])
    album_labels =([a.string for a in album_label])
    album_years = ([a.string for a in album_year])
    times = ([t.string for t in time])
    notes = ([n.string for n in note])

    # create a dictionary of the data
    mydict = {
        'time': times, 
        'artist':artists, 
        'song':songs, 
        'album':albums, 
        'album_label':album_labels, 
        'album_year':album_years, 
        'note':notes,
        'url': url}

    # create a dataframe from the dictionary
    playlist_df = pd.DataFrame(mydict)
  
    # return the dataframe
    return playlist_df

In [ ]:
playlist_df = sound_salvation_spider('https://spinitron.com/KDHX/pl/5319093/Sound-Salvation')
playlist_df


In [ ]:
print(sorted_urls[0])


In [ ]:
playlist_df = sound_salvation_spider('https://spinitron.com/' + sorted_urls[0])
playlist_df

In [ ]:
playlist = []
missing_playlists = []

sound_salvation_total_playlist = pd.DataFrame()


for url in sorted_urls[:]:
    try:
        playlist.append(sound_salvation_spider('https://spinitron.com/' + url))
        sound_salvation_total_playlist = pd.concat(playlist)
        sound_salvation_total_playlist.reset_index(drop=True, inplace=True)

    except ValueError:
        pass
        missing_playlists.append(url) 
        
print('Playlists not included: {}'.format(len(missing_playlists)))
print('Playlists included: {}'.format(len(playlist)))


In [ ]:
sound_salvation_total_playlist

In [ ]:
#output the dataframe to a csv file
# sound_salvation_total_playlist.to_csv('sound_salvation_total_playlist.csv', index=False)

In [ ]:
# https://spinitron.com/KDHX/pl/5410893/Sound-Salvation #is the first url in the list
# https://spinitron.com/KDHX/pl/16398320/Sound-Salvation #most recent

In [ ]:
# look for differences in the spans of the urls
# working url: 'https://spinitron.com/KDHX/pl/5319093/Sound-Salvation'
# url not working: 'https://spinitron.com/KDHX/pl/15311922/Sound-Salvation'

In [ ]:
years = list(range(1930, 2022))

In [ ]:
#figure setting
plt.figure(figsize=(20,10))

plt.xticks(rotation=70)
# plt.set_xticks(years)
plt.xticks(fontsize=13)
plt.title('Sound Salvation Playlist Album Years')
plt.xlabel('Album Year')
plt.ylabel('Count')

#plot
sns.countplot(x='album_year', data=sound_salvation_total_playlist, order=sound_salvation_total_playlist['album_year'].value_counts().index)

In [ ]:
missing_playlists

In [ ]:
#the todos

#find a way to get the playlist urls from the missing_playlists list
#then append them to the total playlist df
#make a date and dj dataframe
#merge the two dataframes
#index by the date
#drop shows with guest dj
#do analysis on the data


In [ ]:
#working:5344299

In [ ]:
def date_and_dj_spider(url):
    """Perform web scraping for any sound salvation playlist given the available link"""
    #get the data
    urls = []
    month = []
    day = []
    year = []
    dj = []
    
    #get the page
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    for page in range(1,35):
   
    url = 'https://spinitron.com/KDHX/show/126906/Sound-Salvation?page={}'.format(page)
    reqs = requests.get(url, headers=headers)
    
    
    rock_soup = BeautifulSoup(reqs.text, 'html.parser')
    rock_soup.prettify()
    
    for link in rock_soup.find_all('a'):
        urls.append(link.get('href'))


   
    dj = rock_soup.find_all('p', attrs={'class':'dj-name'})
    date_stamp = rock_soup.find_all('p', attrs={'class':'timeslot'})
    url = 'https://spinitron.com/' + sorted_urls[0]
    
    # append the data to the empty lists
    djs = ([d.string for d in dj])
    time_slots = ([t.string for t in date_stamp])
    # urls = ([url for u in url])

    # # create a dictionary of the data
    # mydict = {
    #     'url': urls,
    #     'dj': djs,
    #     'time_slot': time_slots
    #     }

    # # create a dataframe from the dictionary
    # playlist_df = pd.DataFrame(mydict)

    print(djs)
    print(time_slots)
    print(url)



In [ ]:
date_and_dj_spider('https://spinitron.com/KDHX/pl/5319093/Sound-Salvation')

In [ ]:
date_dj_df = date_and_dj_spider('https://spinitron.com/' + sorted_urls[0])
date_dj_df


In [ ]:
# worked: </td><td class="spin-text"><div class="spin"><span class="artist">Katrina & The Waves</span> <span class="song">River Deep, Mountain High (Bonus Track)</span></div><div class="info"><span class="release">Katrina and the Waves 2</span> <span class="parenthesis"><span class="label">CGB Records</span> <span class="released">1985</span></span></div><div class="note">A rollicking take on the Ike &amp; Tina Turner/Phil Spector masterpiece.</div>
# didnt:  </td><td class="spin-text"><div class="spin"><span class="artist">Leonard Cohen</span>       <span class="song">Closing Time</span></div><div class="info"><span                            class="release">Live In London</span>          <span class="parenthesis"><span class="label">Columbia Records</span> <span class="released">2009</span></span></div><div class="note">Today's show, which will have plenty of songs he wouldn't necessarily have liked, is dedicated to the memory of Bob Reuter.</div>

In [ ]:
# missing_df = sound_salvation_spider('https://spinitron.com/KDHX/pl/5348184/Sound-Salvation')
# missing_df

In [ ]:
# so look at time
# worked: <td class="spin-time"><a href="/KDHX/pl/5344299/Sound-Salvation?sp=117515787">7:00 AM</a></td>
# didnt:  <td class="spin-time"><a href="/KDHX/pl/5374368/Sound-Salvation?sp=118338438">7:00 AM</a></td>

In [ ]:
def missing_spider(u):
    """Perform web scraping for any sound salvation playlist given the available link"""

    #get the page
    r = requests.get(u)
    rock_soup = BeautifulSoup(r.text, 'html')

    #get the data
    artist = rock_soup.find_all("span", "artist")
    song = rock_soup.find_all("span", "song")
    album = rock_soup.find_all("span", "release")
    album_label = rock_soup.find_all("span", "label")
    album_year = rock_soup.find_all("span", "released")
    time = rock_soup.find_all("td", "spin-time")
    note = rock_soup.find_all('div', attrs={'class':'note'})
    url = u

    # append the data to the empty lists
    artists = ([a.string for a in artist])
    songs = ([s.string for s in song])
    albums = ([a.string for a in album])
    album_labels =([a.string for a in album_label])
    album_years = ([a.string for a in album_year])
    times = ([t.string for t in time])
    notes = ([n.string for n in note])

    #trying to append NONE to the empty lists -- turned out to be unnecessary
    # artists = ([a.string for a in artist] if not a.string.strip() == '' else artists.append(None) for a in artist)
    # songs = ([s.string for s in song] if not s.string.strip() == '' else songs.append(None) for s in song)
    # albums = ([al.string for al in album] if not  al.string.strip() == '' else albums.append(None) for al in album)
    # album_labels = ([alb.string for alb in album_label] if not alb.string.strip() == '' else album_labels.append(None) for alb in album_label)
    # album_years = ([alby.string for alby in album_year] if not alby.string.strip() == ''else album_years.append(None) for alby in album_year)
    # times = ([t.string for t in time] if not t.string.strip() == '' else times.append(None) for t in time)
    # notes = ([n.string for n in note] if not n.string.strip() == '' else notes.append(None) for n in note)

    # create a dictionary of the data
    mydict = {
        'time': times, 
        'artist':artists, 
        'song':songs, 
        'album':albums, 
        'album_label':album_labels, 
        'album_year':album_years, 
        'note':notes,
        'url': url}

    #fill in missing values into mydict with None until the arrays are the same length
    try:
        for key, value in mydict.items():
            if len(value) < len(times):
                value.extend([None]*(len(times)-len(value)))
    except AttributeError:
        pass
        
    # create a dataframe from the dictionary

    # print(len(artist))
    # print(len(song))
    # print(len(album))
    # print(len(album_label))
    # print(len(album_year))
    # print(len(time))
    # print(len(note))
    # print(len(url))

    # print(mydict)
    playlist_df = pd.DataFrame(mydict)
    return playlist_df


In [ ]:
missing_spider('https://spinitron.com/KDHX/pl/5378109/Sound-Salvation')

In [ ]:
#ahhhh! so this is often missing album year or missing notes
#so i need to make sure that the length of the list is the same as the length of the time list
    


In [132]:
playlist = []
missing_playlists = []

sound_salvation_fillin_playlists = pd.DataFrame()


for url in sorted_urls[:]:
    try:
        playlist.append(missing_spider('https://spinitron.com/' + url))
        sound_salvation_fillin_playlists = pd.concat(playlist)
        sound_salvation_fillin_playlists.reset_index(drop=True, inplace=True)

    except ValueError:
        pass
        missing_playlists.append(url) 
        
    print('Playlists not included: {}'.format(len(missing_playlists)))
    print('Playlists included: {}'.format(len(playlist)))

Playlists not included: 0
Playlists included: 1
Playlists not included: 0
Playlists included: 2
Playlists not included: 0
Playlists included: 3
Playlists not included: 0
Playlists included: 4
Playlists not included: 0
Playlists included: 5
Playlists not included: 0
Playlists included: 6
Playlists not included: 0
Playlists included: 7
Playlists not included: 0
Playlists included: 8
Playlists not included: 0
Playlists included: 9
Playlists not included: 0
Playlists included: 10
Playlists not included: 0
Playlists included: 11
Playlists not included: 0
Playlists included: 12


KeyboardInterrupt: 

In [ ]:
sound_salvation_fillin_playlists

In [ ]:
#save to csv
sound_salvation_fillin_playlists.to_csv('sound_salvation_fillin_playlists.csv')

In [133]:

def date_and_dj_spider():
    """Perform web scraping for any sound salvation playlist given the available link"""

urls = []
months = []
days = []
years = []
djs = []

#get the page
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
for page in range(1,35):

    url = 'https://spinitron.com/KDHX/show/126906/Sound-Salvation?page={}'.format(page)
    reqs = requests.get(url, headers=headers)
    
    rock_soup = BeautifulSoup(reqs.text, 'html')
    # rock_soup.prettify()
    
    #get the data 
    for link in rock_soup.find_all("div", {'class':'block'}):

        djs.append(link.find('div', {'class':'dj'}).string)
        months.append(link.find('span', attrs={'class':'month'}).string)
        days.append(link.find('span', attrs={'class':'day'}).string)
        years.append(link.find('span', attrs={'class':'year'}).string)
        urls.append('https://spinitron.com/' + link.find('a').get('href'))

# create a dictionary of the data
mydict = {
    'url': urls,
    'dj': djs,
    'month': months,
    'day': days,
    'year': years
    }

#fill in missing values into mydict with None until the arrays are the same length
try:
    for key, value in mydict.items():
        if len(value) < len(days):
            value.extend([None]*(len(days)-len(value)))
except AttributeError:
    pass
    
print(len(djs))
print(len(months))
print(len(days))
print(len(years))
print(len(urls))


    
# create a dataframe from the dictionary
date_and_dj_df = pd.DataFrame(mydict)


656
656
656
656
656


In [134]:
date_and_dj_df

,url,dj,month,day,year
0,https://spinitron.com//KDHX/pl/16451085/Sound-...,Steve Pick,Sep,23rd,2022
1,https://spinitron.com//KDHX/pl/16424558/Sound-...,Steve Pick,Sep,16th,2022
2,https://spinitron.com//KDHX/pl/16398320/Sound-...,Steve Pick,Sep,9th,2022
3,https://spinitron.com//KDHX/pl/16372205/Sound-...,Steve Pick,Sep,2nd,2022
4,https://spinitron.com//KDHX/pl/16346134/Sound-...,Steve Pick,Aug,26th,2022
...,...,...,...,...,...
651,https://spinitron.com//KDHX/pl/5409969/Sound-S...,Steve Pick,Aug,27th,2010
652,https://spinitron.com//KDHX/pl/5410206/Sound-S...,Steve Pick,Aug,20th,2010
653,https://spinitron.com//KDHX/pl/5410428/Sound-S...,Steve Pick,Aug,13th,2010
654,https://spinitron.com//KDHX/pl/5410671/Sound-S...,Steve Pick,Aug,6th,2010


In [135]:
#make a new column for the date in the date_and_dj_df as a datetime object
date_and_dj_df['date'] = pd.to_datetime(date_and_dj_df['month'] + ' ' + date_and_dj_df['day'] + ' ' + date_and_dj_df['year'])
#drop the month, day, and year columns
date_and_dj_df.drop(['month', 'day', 'year'], axis=1, inplace=True)
#set the index to the date
# date_and_dj_df.set_index('date', inplace=True)

date_and_dj_df


,url,dj,date
0,https://spinitron.com//KDHX/pl/16451085/Sound-...,Steve Pick,2022-09-23
1,https://spinitron.com//KDHX/pl/16424558/Sound-...,Steve Pick,2022-09-16
2,https://spinitron.com//KDHX/pl/16398320/Sound-...,Steve Pick,2022-09-09
3,https://spinitron.com//KDHX/pl/16372205/Sound-...,Steve Pick,2022-09-02
4,https://spinitron.com//KDHX/pl/16346134/Sound-...,Steve Pick,2022-08-26
...,...,...,...
651,https://spinitron.com//KDHX/pl/5409969/Sound-S...,Steve Pick,2010-08-27
652,https://spinitron.com//KDHX/pl/5410206/Sound-S...,Steve Pick,2010-08-20
653,https://spinitron.com//KDHX/pl/5410428/Sound-S...,Steve Pick,2010-08-13
654,https://spinitron.com//KDHX/pl/5410671/Sound-S...,Steve Pick,2010-08-06


In [136]:
#save date_and_dj_df to csv
date_and_dj_df.to_csv('date_and_dj_df.csv')

In [ ]:
#merge the two dataframes, fill forward the dj column
sound_salvation_fillin_playlists = pd.merge(sound_salvation_fillin_playlists, date_and_dj_df, on='url', how='left')
sound_salvation_fillin_playlists['dj'].fillna(method='ffill', inplace=True)


In [ ]:
sound_salvation_fillin_playlists